In [1]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window

In [2]:
# Set root dir
os.chdir("/")
print("Current Working Directory:", os.getcwd())

# !!CHANGE THIS!!
working_dir="/BigData"

dataset_dir=f"{working_dir}/dataset"

dataset_original_csv = f'{dataset_dir}/creditScores.csv'

dataset_csv_path = f'{dataset_dir}/creditScores_csv'
dataset_json_path = f'{dataset_dir}/creditScores_json'
dataset_parquet_path = f'{dataset_dir}/creditScores_parquet'
dataset_orc_path = f'{dataset_dir}/creditScores_orc'
dataset_avro_path = f'{dataset_dir}/creditScores_avro'

clean_dataset_path = f'{dataset_dir}/clean_creditScores_orc'

os.chdir(working_dir)
print(f"New working directory: {working_dir}")


Current Working Directory: /
New working directory: /BigData


In [3]:
proj_conf = SparkConf()\
        .setAppName("projBigData_preProcess")\
        .set("spark.driver.memory", "16g")\
        .set("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0")

# Create a Spark session using the configured SparkConf
spark = SparkSession.builder.config(conf=proj_conf).getOrCreate()


In [4]:
# Read ORC files into a Spark DataFrame
df = spark.read.orc(dataset_orc_path)
df.orderBy("Customer_ID").show()

+-------+-----------+--------+--------------+---+-----------+-------------+------------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+------------------+------------+
|     ID|Customer_ID|   Month|          Name|Age|        SSN|   Occupation|     Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|   Monthly_Balance|Credit_Score|
+-------+-----------+--------+--------------+---

In [5]:
count_rows = df.count()
df.printSchema()
# Prints column information and number of rows
print("Number of rows in the DataFrame:", count_rows)

root
 |-- ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Annual_Income: string (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: integer (nullable = true)
 |-- Num_Credit_Card: integer (nullable = true)
 |-- Interest_Rate: integer (nullable = true)
 |-- Num_of_Loan: string (nullable = true)
 |-- Type_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: string (nullable = true)
 |-- Changed_Credit_Limit: string (nullable = true)
 |-- Num_Credit_Inquiries: double (nullable = true)
 |-- Credit_Mix: string (nullable = true)
 |-- Outstanding_Debt: string (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Credit_History_Age: string (nullable = true

# Functions
The following functions prepare the data to a minimum standard that will allow us to then analyse the data withouth having to frequently change types and fill empty/null values

## Replace by most common
This function allows us to populate missing data from the same Customer_ID with precision, given that some data is universal to the same Customer independently of the data entry. As such we can retain more of the original dataset without having to drop a substantial amount of rows 

In [6]:
def replace_by_most_common_value_in_group(df, column, group_col="Customer_ID"):
    # Define the window specification over the group_col
    windowSpec = Window.partitionBy(group_col)
    
    # Count the occurrences of each value in the specified column within each group
    value_counts = df.groupBy(group_col, column).count()
    
    # Find the most common value for each group
    most_common_value = (value_counts
                         .withColumn('max_count', F.max('count').over(windowSpec))
                         .where(F.col('count') == F.col('max_count'))
                         .drop('count', 'max_count')
                         .dropDuplicates([group_col])
                         .withColumnRenamed(column, f"{column}_common"))

    # Join the most common values back to the original DataFrame
    df = df.join(most_common_value, [group_col], 'left_outer')
    
    # Replace the original column values with the most common values
    df = df.withColumn(column, 
                       F.when(df[f"{column}_common"].isNotNull(), df[f"{column}_common"])
                       .otherwise(df[column])
                      ).drop(f"{column}_common")
    
    return df

## Casts negative numbers
In order to deal with negative numbers in columns that should not have negatives, such as Age, we replace those values with NaN, so the data can be plotted.
The NaN values are then handled on the data processing.

In [7]:
def cast_numeric_no_negatives(df, col_name):
    # Verifying if the column exists in the DataFrame
    if col_name in df.columns:
        # Applying the function to the column to replace negative values with NaN
        df = df.withColumn(col_name, 
                           F.round(F.col(col_name), 4).cast("float").alias(col_name))
        df = df.withColumn(col_name, 
                           F.when(F.col(col_name) >= 0, F.col(col_name)).otherwise(None))
    else:
        print("Column not found in DataFrame.")
    return df

## Cast Numeric
Converts column numeric type to float

In [8]:
def cast_numeric(df, col_name):
    # Verifying if the column exists in the DataFrame
    if col_name in df.columns:
        # Convert the column to a numeric type and handle non-numeric values
        df = df.withColumn(col_name, F.when(F.col(col_name).cast("float").isNull(), F.lit(None)).otherwise(F.round(F.col(col_name).cast("float"), 4)))
    else:
        print("Column not found in DataFrame.")
    return df

## Cast Integer
Converts column numeric type to integer

In [9]:
def cast_int(df, column):
    # Convert the column to IntegerType and handle errors with null values
    df = df.withColumn(column, F.col(column).cast(IntegerType()))
    return df

## Replace with empty string
Replaces unwanted strings of characters with an empty string

In [10]:
def replace_by_empty(df, column, value_to_replace):
    # Replace specified values with empty string (null)
    df = df.withColumn(column, F.when(F.col(column) == value_to_replace, '').otherwise(F.col(column)))
    return df

# Pre-Process
Call functions to process specific column groups

In [11]:
# Cast numeric columns without allowing negative values
numeric_cols_no_negatives = ["Age", "Annual_Income", "Monthly_Inhand_Salary", "Num_Bank_Accounts",
                             "Num_Credit_Card", "Interest_Rate", "Num_of_Loan", "Delay_from_due_date",
                             "Num_of_Delayed_Payment", "Num_Credit_Inquiries", "Outstanding_Debt",
                             "Credit_Utilization_Ratio", "Total_EMI_per_month", "Amount_invested_monthly",
                             "Monthly_Balance"]
for col in numeric_cols_no_negatives:
    df = cast_numeric_no_negatives(df, col)

numeric_cols = ["Changed_Credit_Limit"]
for col in numeric_cols:
    df = cast_numeric(df, col)

# Cast specific columns to IntegerType
int_values = ["Age", "Num_Bank_Accounts", "Num_Credit_Card", "Num_of_Loan", "Delay_from_due_date", "Num_Credit_Inquiries"]
for col in int_values:
    df = cast_int(df, col)

# Replace values in specified columns by the most common value within each group of "Customer_ID"
uniform_by_client = ["Name", "SSN", "Occupation", "Annual_Income"]
for col in uniform_by_client:
    df = replace_by_most_common_value_in_group(df, col, "Customer_ID")

# Replace specific values in columns with predefined replacements
df = replace_by_empty(df, "Payment_Behaviour", "!@9#%8")
df = replace_by_empty(df, "Occupation", "_______")
df = replace_by_empty(df, "Credit_Mix", "_")
df = replace_by_empty(df, "Payment_of_Min_Amount", "NM")
df = replace_by_empty(df, "Credit_History_Age", "NA")

In [12]:
df.orderBy("Customer_ID").show()

+-----------+-------+--------+--------------+----+-----------+-------------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+---------------+------------+
|Customer_ID|     ID|   Month|          Name| Age|        SSN|   Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|Credit_Score|
+-----------+-------+--------+--------------+----+-----------+

In [13]:
df.show()
count_rows = df.count()
df.printSchema()
print("Number of rows in the DataFrame:", count_rows)

+-----------+------+--------+----------------+----+-----------+-------------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+---------------+------------+
|Customer_ID|    ID|   Month|            Name| Age|        SSN|   Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|Credit_Score|
+-----------+------+--------+----------------+----+---------

In [14]:
# Extract years and months separately using regular expressions
years = F.regexp_extract(F.col("Credit_History_Age"), r"(\d+) Years", 1).cast("int")
months = F.regexp_extract(F.col("Credit_History_Age"), r"(\d+) Months", 1).cast("int")

# Convert years to months and add with months
total_months = years * 12 + months

# Add the new column to the DataFrame
df = df.withColumn("Credit_History_Age_Months", total_months)

df.show()

+-----------+------+--------+----------------+----+-----------+-------------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+---------------+------------+-------------------------+
|Customer_ID|    ID|   Month|            Name| Age|        SSN|   Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|Credit_Score|Credit_History_Age_Months|
+-------

In [15]:
# Convert hexadecimal 'ID' to decimal (integer)
df = df.withColumn("ID_decimal", F.conv(F.expr("substring(ID, 3)"), 16, 10).cast("int"))
df.show()

+-----------+------+--------+----------------+----+-----------+-------------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+---------------+------------+-------------------------+----------+
|Customer_ID|    ID|   Month|            Name| Age|        SSN|   Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|Credit_Score|Credit_History_Age_Month

In [16]:
# Remove 'CUS_' prefix from 'Customer_ID'
df = df.withColumn("Customer_ID", F.regexp_replace(F.col("Customer_ID"), "CUS_", ""))

# Convert hexadecimal 'Customer_ID' to decimal (integer)
df = df.withColumn("Customer_ID_decimal", F.conv(F.expr("substring(Customer_ID, 3)"), 16, 10).cast("int"))
df.show()

+-----------+------+--------+----------------+----+-----------+-------------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+---------------+------------+-------------------------+----------+-------------------+
|Customer_ID|    ID|   Month|            Name| Age|        SSN|   Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|Credit_Score|Cred

In [17]:
# Drop specified columns (to be substituted by 'ID_decimal', 'Customer_ID_decimal','Credit_History_Age_Months')
df = df.drop('ID', 'Customer_ID','Credit_History_Age')

# Define the desired column order
desired_order = [ 
    'ID_decimal', 'Customer_ID_decimal',
    'Month', 'Name', 'Age', 'SSN', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary',
    'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
    'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
    'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio',
    'Credit_History_Age_Months', 'Payment_of_Min_Amount', 'Total_EMI_per_month',
    'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score',    
]

In [18]:
# Reorder columns as per 'desired_order'
df = df.select(*desired_order)

# Rename 'ID_decimal' and 'Customer_ID_decimal' back to 'ID' and 'Customer_ID'
df = df.withColumnRenamed('ID_decimal', 'ID').withColumnRenamed('Customer_ID_decimal', 'Customer_ID').withColumnRenamed('Credit_History_Age_Months', 'Credit_History_Age')     

# Show the updated DataFrame
df.show()

+-----+-----------+--------+----------------+----+-----------+-------------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+------------------+---------------------+-------------------+-----------------------+--------------------+---------------+------------+
|   ID|Customer_ID|   Month|            Name| Age|        SSN|   Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|Credit_Score|
+-----+-----------+--------+----------------+----+-----------+----

In [19]:
df.orderBy("Customer_ID").show()

+-----+-----------+--------+-----------------+----+-----------+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+------------------+---------------------+-------------------+-----------------------+--------------------+---------------+------------+
|   ID|Customer_ID|   Month|             Name| Age|        SSN|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|Credit_Score|
+-----+-----------+--------+-----------------+----+-----------+-------

In [20]:
x = df.toPandas()
 
x.to_csv('/BigData/dataset/clean.csv', index=False)

In [21]:
# Save DB
df = df.repartition(16)
df.write.mode("overwrite").orc(clean_dataset_path)

In [23]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Customer_ID: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SSN: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Annual_Income: float (nullable = true)
 |-- Monthly_Inhand_Salary: float (nullable = true)
 |-- Num_Bank_Accounts: integer (nullable = true)
 |-- Num_Credit_Card: integer (nullable = true)
 |-- Interest_Rate: float (nullable = true)
 |-- Num_of_Loan: integer (nullable = true)
 |-- Type_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: float (nullable = true)
 |-- Changed_Credit_Limit: float (nullable = true)
 |-- Num_Credit_Inquiries: integer (nullable = true)
 |-- Credit_Mix: string (nullable = true)
 |-- Outstanding_Debt: float (nullable = true)
 |-- Credit_Utilization_Ratio: float (nullable = true)
 |-- Credit_History_Age: integer (nullable = true)


In [22]:
spark.stop()